In [1]:
import numpy as np

In [2]:
def load_data(h,size):
    if h == 1:
        data = np.loadtxt('h'+str(size)+'.txt')
    elif h == 0:
        data = np.loadtxt('l'+str(size)+'.txt')
    else:
        print('Error.')
    proftis = data[:,3]
    data = data[:,1:3]
    return data,proftis

In [3]:
data = None
profits = None
disMat = None
inner_epochs = 500
meta_epochs = 5000
size = None

In [4]:
def modification(tour):
    #return np.random.permutation(tour)
    return tour
def initial_tour(size):
    l = int(size/4)
    tour = np.zeros((l+1))
    tour[0] = 0
    tour[l] = 0
    tour[1:l] = np.random.choice(size-1,l-1,replace=False)+1
    return tour
def tour_cost(tour):
    global disMat,profits
    tour_length = len(tour)
    cost = 0
    for i in range(tour_length-1):
        cost += disMat[int(tour[i]),int(tour[i+1])]
        cost -= profits[i]
    return cost
def simulated_annealing(tour,epochs,cost):
    length = len(tour)
    Tinit = 5
    Tfinal = 1
    for k in range(epochs):
        T = np.power(Tinit*(Tfinal/Tinit),k/epochs)
        if length < 4:
            return tour,cost
        choices = np.random.choice(length-2,2,replace=False);
        choices = choices +1
        i = min(choices)
        j = max(choices)
        new_tour = np.copy(tour);
        new_tour[i:j+1] = tour[i:j+1][::-1];
        new_cost = tour_cost(new_tour)
        delta = new_cost - cost
        p = min(np.exp(-delta/T),1)
        if np.random.rand() < p:
            tour = np.copy(new_tour)
            cost = new_cost
    return np.copy(tour),cost
def insert(tour):
    length = len(tour)
    if length == size+1:
        return tour,tour_cost(tour)
    else:
        j = np.random.choice(np.setdiff1d(range(1,size),tour),1)
        new_tour = np.zeros(length+1)
        shuffle = np.copy(tour[1:length-1])
        shuffle = np.insert(shuffle,0,j)
        new_tour[1:length] = modification(shuffle)
        new_cost = tour_cost(new_tour)
        return simulated_annealing(new_tour,inner_epochs,new_cost)
def remove(tour):
    length = len(tour)
    if length == 2:
        return tour,tour_cost(tour)
    j = np.random.choice(length-2,1)+1
    new_tour = np.delete(tour,j)
    new_tour[1:length-2] = modification(new_tour[1:length-2])
    new_cost = tour_cost(new_tour)
    return simulated_annealing(new_tour,inner_epochs,new_cost)

In [5]:
def meta_sa(s):
    global profits,data,disMat,size
    size = s
    data,profits = load_data(0,size)
    disMat = np.zeros((size,size))
    for i in range(size):
        for j in range(size):
            disMat[i,j] = np.linalg.norm(data[i,:] - data[j,:])
    tour = initial_tour(size)
    cost = tour_cost(tour)
    tour,cost = simulated_annealing(tour,15000,cost)
    length = len(tour)
    print(cost,length)
    Tinit = 20
    Tfinal = 1
    for k in range(1,meta_epochs):
        if k % 100 == 0:
            print(k,cost,len(tour))
        T = np.power(Tinit*(Tfinal/Tinit),k/meta_epochs)
        choices = np.random.choice(length-2,2,replace=False);
        choices = choices +1
        choices = choices +1
        i = min(choices)
        j = max(choices)
        if np.random.rand() > 0.2:
            new_tour,new_cost = insert(tour)
        else:
            new_tour,new_cost = remove(tour)
        delta = new_cost - cost
        p = min(np.exp(-delta/T),1)
        if np.random.rand() < p:
            tour = np.copy(new_tour)
            cost = new_cost
    print(cost,len(tour))

In [ ]:
for i in [51,76,101]:
    print('Low Profit  ',str(i))
    meta_sa(i)

Low Profit   51
74.02291420064994 13
100 1.024154793486428 20
200 2.392681076573594 21
300 0.40408258796725605 20
400 -4.641753222062348 20
500 -4.641753222062348 20
600 -6.055331903550577 20
700 -6.055331903550577 20
800 -3.2852431605168366 20
900 -3.2852431605168366 20
1000 -3.2852431605168366 20
1100 -3.2852431605168366 20
1200 -3.2852431605168366 20
1300 -2.7904625413939517 20
1400 -10.865384925503031 20
1500 -10.865384925503026 20
1600 -10.68771004142835 20
1700 -14.353521620284006 20
1800 -14.353521620284006 20
1900 -14.353521620284006 20
2000 -14.565672605652761 20
2100 -14.565672605652761 20
2200 -14.565672605652761 20
2300 -14.565672605652761 20
2400 -14.565672605652761 20
2500 -14.565672605652761 20
2600 -14.565672605652761 20
2700 -14.565672605652761 20
2800 -14.565672605652761 20
2900 -14.565672605652761 20
3000 -14.565672605652761 20
3100 -14.565672605652761 20
3200 -14.565672605652761 20
3300 -14.565672605652761 20
3400 -14.565672605652761 20
3500 -14.69513821944869 20
36